In [2]:
import requests
import pandas as pd
import numpy as np
import time
import json
from google.oauth2.service_account import Credentials
from google.cloud import bigquery

# TODO: Set path to credentials 
cred_path = r'C:\Users\vicow\Documents\PlaceSense\credentials_cs.json'
credentials = Credentials.from_service_account_file(cred_path)

# Create GBQ client
bqclient = bigquery.Client(credentials=credentials)

In [6]:
coordinates = pd.read_csv('coordinates/den_haag.csv')

locations = []
for i,j in zip(coordinates['avg_longitude'], coordinates['avg_latitude']):
    locations.append([i,j])
locations = locations[:10]

locations

[[4.378308747355544, 52.06905274939536],
 [4.289123111155115, 52.07042700637442],
 [4.277123128301878, 52.09660214513916],
 [4.286990047163462, 52.06416877556382],
 [4.280999322209033, 52.04303419899175],
 [4.29965745028815, 52.090771438049536],
 [4.267167041204614, 52.0515162334313],
 [4.298759130910276, 52.01527408010378],
 [4.365983846451943, 52.0955401845543],
 [4.279287121964956, 52.078123824787255]]

In [9]:
sql = """
    SELECT 
        level, country_1, entity_type_1, entity_code_1,
        ARRAY_AGG(entity_code_2) entity_code_2
    FROM `gifted-slice-218906.production.geolocation_distances`
    LEFT JOIN (SELECT level level_1, aerial_distance_km_filter FROM `gifted-slice-218906.production.geolocation_levels`) ON level_1 = level
    WHERE level = 1 AND aerial_distance < aerial_distance_km_filter
    GROUP BY level, country_1, entity_type_1, entity_code_1
    LIMIT 10000
"""

geolocations = bqclient.query(sql).result().to_dataframe()
geolocations.head()

C:\Users\vicow\AppData\Local\Temp\ipykernel_13756\3895679366.py:12: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  geolocations = bqclient.query(sql).result().to_dataframe()


,level,country_1,entity_type_1,entity_code_1,entity_code_2
0,1,at,statistical_area,31026008,"[31321003, 32135002, 31104000, 31304000, 31111..."
1,1,at,statistical_area,31207003,"[91301110, 92301141, 31952004, 92301050, 92301..."
2,1,at,statistical_area,40101342,"[40802001, 41808002, 41807000, 40827000, 40831..."
3,1,at,statistical_area,30604002,"[92201131, 32131005, 92201134, 31921000, 92101..."
4,1,at,statistical_area,32305003,"[92301013, 92301093, 90501025, 90501029, 91201..."


In [20]:
sql = f"""
    SELECT entity_code entity_code_1, avg_longitude avg_longitude_1, avg_latitude avg_latitude_1 FROM `gifted-slice-218906.production.geolocations`
    WHERE entity_code in UNNEST({list(geolocations['entity_code_1'])})
"""

df = bqclient.query(sql).result().to_dataframe()
df.head()

C:\Users\vicow\AppData\Local\Temp\ipykernel_13756\3559004539.py:6: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  df = bqclient.query(sql).result().to_dataframe()


,entity_code_1,avg_longitude_1,avg_latitude_1
0,BU02130607,6.072187,52.100423
1,2826,5.609711,51.589717
2,BU18760004,6.290135,52.029751
3,BU07430100,5.762970,51.384869
4,BU07960912,5.302515,51.710669


In [21]:
geolocations = pd.merge(left=geolocations, right=df, how='left', on='entity_code_1')
geolocations

,level,country_1,entity_type_1,entity_code_1,entity_code_2,avg_longitude,avg_latitude,avg_longitude_1,avg_latitude_1
0,1,at,statistical_area,31026008,"[31321003, 32135002, 31104000, 31304000, 31111...",15.858162,48.597422,15.858162,48.597422
1,1,at,statistical_area,31207003,"[91301110, 92301141, 31952004, 92301050, 92301...",16.417624,48.412800,16.417624,48.412800
2,1,at,statistical_area,40101342,"[40802001, 41808002, 41807000, 40827000, 40831...",14.313198,48.271267,14.313198,48.271267
3,1,at,statistical_area,30604002,"[92201131, 32131005, 92201134, 31921000, 92101...",16.238680,48.008278,16.238680,48.008278
4,1,at,statistical_area,32305003,"[92301013, 92301093, 90501025, 90501029, 91201...",16.266005,47.885716,16.266005,47.885716
...,...,...,...,...,...,...,...,...,...
20657,1,nl,neighborhood,BU03530007,"[BU06100210, BU03639604, BU04370008, BU0484050...",5.055307,52.026581,5.055307,52.026581
20658,1,nl,neighborhood,BU03580000,"[BU19310806, BU04060204, BU05130203, BU1916020...",4.751154,52.270268,4.751154,52.270268
20659,1,nl,neighborhood,BU03561500,"[BU06100206, BU19420300, BU19420100, BU0610020...",5.084093,52.017809,5.084093,52.017809
20660,1,nl,neighborhood,BU03520005,"[BU05120403, BU04020601, BU05121600, BU1696040...",5.348139,51.977899,5.348139,51.977899


In [ ]:
body = {"locations":locations,"metrics":["distance", "duration"],"sources":[1]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624867d60a4709b54dbeb853215f4841061a',
    'Content-Type': 'application/json; charset=utf-8'
}

start = time.time()
call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=body, headers=headers)
result = json.loads(call.text)
end = time.time()

if call.status_code == 200:
    print("\nRequest successfull with response " + str(call.status_code))
    print("\nNumber of geolocations: " + str(len(locations)))
    print("Execution time: " + str(round(end-start, 3)) + " seconds")
    distances = np.array(result['distances'])
    print("Resulting matrix of shape: " + str(distances.shape))
else:
    print("\nRequest failed with response " + str(call.status_code))
    print("\nError: " + str(result['error']))